In [1]:
import pickle
from latain.comm.data import CommCollator

In [2]:
with open("data/comm_records_v2.pkl", "rb") as f:
    records = pickle.load(f)

In [3]:
tags = ["O", "timeRange", "company", "title", "depart", "area", "salary"]
collator = CommCollator(tags, 64)

In [4]:
from moka_tokenizer import moka_codec

In [5]:
batch = collator([records[1]])

In [6]:
for idx, span in enumerate(records[1].spans):
    span_text = "".join(span.tokens)
    print(f"{idx}\t{span_text}")

0	最近工作
1	最高学历/学位
2	职位:
3	行政人事专员/助理
4	专业:
5	计算机应用
6	公司:
7	大苑子企业管理咨询(上海)有限公司
8	学校:
9	苏州五二六技校
10	行业:
11	快速消费品(食品,饮料,化妆品)
12	学历/学位:
13	中专
14	工作经验
15	2017/4-至今
16	行政人事专员/助理|行政管理部
17	大苑子企业管理咨询(上海)有限公司[ 2年1个月]
18	快速消费品(食品,饮料,化妆品) | 50-150人|外资(欧美)
19	工作描述:
20	1.负责公司会议室管理,使用,维护
21	2.负责办公室采购,入库,领用
22	3.负责办公室办公用品维护,保修
23	4.负责上海总部的员工花名册,档案管理,员工入职,离职手续办理
24	5.负责3个招聘网站管理
25	6.负责每月员工考勤,工资初步计算
26	7.负责公司所有合同管理
27	8.负责台湾全球会议及加盟商的会议事宜
28	5.负责台湾总部的文件签核,录入,归档
29	9.负责意向加盟者的电话和后期追踪
30	10.负责公司内店的日常报表及销售情况
31	11.负责公司pos,erp,微会员的系统管理
32	离职原因:公司遣散
33	2015/1-2017/3
34	总裁助理/行政人事专员|行政部
35	上海丰邦建材科技有限公司[ 2年2个月]
36	建筑/建材/工程| 50-150人|民营公司
37	工作描述:
38	人事行政助理工作:
39	1.负责每月考勤统计及结算工资,各类请假的统计,审批o
40	2.负责员工的档案归类,入职,离职的手续办理o
41	3.负责对新员工进行企业制度与文化的培训工作,建立企业的形象o
42	4.负责对公司合作的招聘网站的管理,进行招聘,初试及安排上班o
43	5.负责按照领导要求修改或拟定企业的行政,人事制度o
44	6.负责公司的办公用品及日用品的申购,入库和发放,申报采购款o
45	7.负责公司各类文件,资料下发,张贴及登记归档o
46	8.负责后勤车辆的管理,钥匙保管,记录车辆的违章,保险及验车情况o
47	9.负责相关部门的申请款费,报销,统计o
48	10.负责公司宿舍的安排入住及监督卫生o
49	总裁行政助理:
50	1.负责公司的各类会议的安排及通知工作o
51	2.负责与各分公司的对接工作,配合各分公司

In [10]:
span_idx = 59
span_tokens = batch["input_ids"][span_idx]
span_text = moka_codec.decode([i for i in span_tokens if i])
span_tags = batch["tags"][span_idx]
span_tag_text = " ".join(collator.int2tag(i) if i != -100 else "" for i in span_tags)

similar_span_idx = batch["comm_matrix"][span_idx].argsort(descending=True)[:3].tolist()
similar_scores = [batch["comm_matrix"][span_idx][i].item() for i in similar_span_idx]

print(f"span_text:\n{span_text}")
print(f"span_tag_text:\n{span_tag_text}")
for score, idx in zip(similar_scores, similar_span_idx):
    span_tokens = batch["input_ids"][idx]
    span_text = moka_codec.decode([i for i in span_tokens if i])
    print(score)
    print(span_text)

span_text:
<CLS> 店经理 ( 红酒 ) | 店经理 <SEP>
span_tag_text:
 title title title title title title title title depart depart depart                                   
0.32451155967520884
<CLS> 行政人事专员 / 助理 | 行政管理部 <SEP>
0.3030337525353308
<CLS> 总裁助理 / 行政人事专员 | 行政部 <SEP>
0.2672879420321419
<CLS> 泊美专柜小姐 | 销售 <SEP>


In [26]:
similar_span_idx
similar_scores

[0.96162059540844, 0.9289525850240321, 0.9252234969532699]

In [27]:
comm_matrix = batch["comm_matrix"]
comm_matrix[comm_matrix < 0.1] = 0

In [28]:
a = comm_matrix  / comm_matrix.sum(dim=-1)

In [29]:
[a[span_idx][i].item() for i in similar_span_idx]

[0.20531298888925456, 0.20121724671702607, 0.20026537538539838]

In [31]:
similar_span_idx

[33, 79, 70]

In [32]:
for idx in similar_span_idx:
    span_text = moka_codec.decode(batch["input_ids"][idx])
    print(span_text)

<CLS> 2 0 1 5 / 1 - 2 0 1 7 / 3 <SEP> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
<CLS> 2 0 0 6 / 8 - 2 0 1 0 / 1 <SEP> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
<CLS> 2 0 1 1 / 9 - 2 0 1 3 / 1 0 <SEP> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


In [20]:
a[span_idx]

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0679, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0681, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0676, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0671, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0674, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0672, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0673, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0669, 0.0000, 

In [33]:
batch["comm_matrix"].sum(dim=-1)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 

In [29]:
similar_span_idx

[110, 36, 121]